# Using OpenCV to Reorder Images based on Point Tags
## Import libraries

In [21]:
import matplotlib as plt
import numpy as np
import cv2 as cv
import os
import pathlib

In [22]:
build_path = pathlib.Path("Build/").mkdir(exist_ok=True)
source_path = pathlib.Path("Test Data/fumo/")

#Setup Files for reading
puzzle_pieces = []
with os.scandir(source_path) as entries:
    for entry in entries:
        puzzle_pieces.append(entry.name)

In [23]:
#Store answers
position_answers = []

In [24]:
# Set Marker Color Ranges 
# Note that CV uses BGR instead of RGB
blue_dot = [(200,0,0),(255,40,40),"blue"]
red_dot = [(0,0,200),(40,40,255),"red"]
markers = [blue_dot,red_dot]

In [25]:
def loader(image_filename:str):
    return cv.imread(str(source_path.joinpath(image_filename)))

In [26]:
def solve_position(image_filename:str ) -> dict:
    # test_image = source_path.joinpath(image_filename)
    # loaded_image = cv.imread(str(test_image))
    loaded_image = loader(image_filename)

    #Pre=process image
    blurred_image = cv.medianBlur(loaded_image, 7)

    #Populate total markers in image to Dictionary
    circles = {}
    for lower_bound, upper_bound, color in markers:
        mask = cv.inRange(blurred_image,lower_bound, upper_bound)
        # cv.imshow('mask',mask)
        
        counts = cv.findContours(mask,cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)
        circles.update({color : len(counts[0])})
        # cv.destroyWindow('mask')
    
    # cv.destroyAllWindows()
        
    return circles

#Test function works as intended        
assert( solve_position("3Yy2JsTKtsLVG2RU.jpg") == {'blue':2, 'red':2})
    


Columns are Red, Rows are Blue

In [27]:
#go through set of images
for image in puzzle_pieces:
    position = solve_position(image)
    solved_piece = position['red'], position['blue'], image
    position_answers.append(solved_piece)


In [28]:

#find the max row and column
row_size = 0
col_size = 0
for answer in position_answers:
    col_size = max(answer[0],col_size)
    row_size = max(answer[1],row_size)
    
#find shape of total 
single_shape = loader(puzzle_pieces[0]).shape
max_x = col_size * single_shape[0] 
max_y = row_size * single_shape[1] 

In [29]:
row_size, col_size

(3, 3)

In [30]:
max_x, max_y

(600, 600)

In [35]:
# Assemble picture
# puzzle_area = np.zeros((max_x, max_y,3),dtype='uint8')

def firstElement(element):
    return element[0]

position_answers.sort(key=firstElement)
horizontal_strip = []
vertical_strip = []
for x,y,image in position_answers:
    if (x <= col_size):
        horizontal_strip.append(image)
    else:
        width = np.concatenate(list(map(loader,horizontal_strip)),axis=1)
        horizontal_strip.clear()

    # np.concatenate(vertical_strip, axis=0)
        
print(position_answers)
# for i in position_answers:
#     print(i)
#     grid_pt_x = max_x/col_size
#     grid_pt_y = max_y/row_size
#     point1_x = (i[0]-1) * grid_pt_x
#     point1_y = (i[1]-1) * grid_pt_y
#     point2_x = point1_x + grid_pt_x
#     point2_y = point1_y + grid_pt_y
    
#     print(point1_x,point1_y)
#     print(point2_x, point2_y)
#     puzzle_section = loader(i[2])
    # cv.rectangle(puzzle_section, (point1_x,point1_y),(point2_x, point2_y),)
# cv.imshow('Puzzle',puzzle_area)
# cv.waitKey(delay=0)

[(1, 1, 'jeKu7769tCCqm5NG.jpg'), (1, 3, 'O1f0qQidDoOgejBm.jpg'), (1, 2, 'so1rDdxWjmh8zs7K.jpg'), (2, 2, '3Yy2JsTKtsLVG2RU.jpg'), (2, 3, 'J21Sb3z7N2KwqviC.jpg'), (2, 1, 'XLN7viVqN92pKe14.jpg'), (3, 1, 'g8B4OJ6a0y8agQmK.jpg'), (3, 2, 'ghsavmJgmo6kLOY3.jpg'), (3, 3, 'RnfFLB1SsZMsimeA.jpg')]
